In [1]:
import pandas as pd
df1=pd.read_csv(r"C:\Users\hp\Desktop\internship\search engine project\cleand.csv")
df1.head()

,Ids,title,subtitles
0,9422324,doctor who confidential s06 e11 heartbreak hot...,this is matt smithconfidential primetime. you...
1,9200952,american experience s16 e06 tupperware 2004,film music plays film announceramerican busi...
2,9481570,the last nosferatu 2023,support us and become vip member to remove al...
3,9431169,the long way home 1997,advertise your product or brand herecontact t...
4,9227619,ncis s06 e02 agent afloat 2008,agent afloatreport to the fantail. agent aflo...


In [2]:
import chromadb
from chromadb.utils import embedding_functions
client = chromadb.PersistentClient(path="vectortdb")
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import DataLoader

model='paraphrase-MiniLM-L3-v2'


collection=client.get_or_create_collection(name='subtitle',
                                    metadata={'hnsw':'cosine'},
                                    )#device='cuda' embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model)

c:\Users\hp\Desktop\internship\search engine project\.env_subtitles\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
client.delete_collection('subtitles')

In [4]:
! pip install transformers -q

You should consider upgrading via the 'C:\Users\hp\Desktop\internship\search engine project\.env_subtitles\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
def document_chunker(data, chunk_size=500, overlap_size=50):
    chunks = []
    start_idx = 0
    while start_idx < len(data):
        end_idx = min(start_idx + chunk_size, len(data))
        chunk = ' '.join(data[start_idx:end_idx])
        chunks.append(chunk)
        start_idx += chunk_size - overlap_size
    return chunks

In [7]:
chunked_texts = document_chunker(df1['subtitles'].tolist())

In [12]:
def generate_embeddings(texts):
    model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
    embeddings = []
    for text_chunk in texts:
        chunk_embeddings = model.encode(text_chunk)
        embeddings.append(chunk_embeddings)
    return embeddings

In [13]:
chunk_embeddings=generate_embeddings(chunked_texts)

In [15]:
for index, row in df1.iterrows():
    document_id = str(row['Ids'])  # Use a unique identifier for each document
    document_text = row['subtitles']
    # document_embedding = corpus_embeddings[index % len(corpus_embeddings)]  # Assuming embeddings are aligned with the rows

    # ------------------------------------------------------------------------
    chunk_index = index % len(chunk_embeddings)
    document_embedding = chunk_embeddings[chunk_index]
    #--------------------------------------------------------------------------
    metadata = {'Title': row['title']}

    # Insert document into ChromaDB collection
    collection.add(ids=document_id, documents=[document_text], embeddings=[document_embedding.tolist()], metadatas=[metadata])


print("Insertion into ChromaDB collection complete")

Add of existing embedding ID: 9422324
Insert of existing embedding ID: 9422324
Add of existing embedding ID: 9200952
Insert of existing embedding ID: 9200952
Add of existing embedding ID: 9481570
Insert of existing embedding ID: 9481570
Add of existing embedding ID: 9431169
Insert of existing embedding ID: 9431169
Add of existing embedding ID: 9227619
Insert of existing embedding ID: 9227619
Add of existing embedding ID: 9351195
Insert of existing embedding ID: 9351195
Add of existing embedding ID: 9431748
Insert of existing embedding ID: 9431748
Add of existing embedding ID: 9344584
Insert of existing embedding ID: 9344584
Add of existing embedding ID: 9431471
Insert of existing embedding ID: 9431471


Insertion into ChromaDB collection complete


In [19]:
query='the last nosferatu 2023'
collection.query(
    query_texts = query,
    include=["metadatas", 'distances'],
    n_results=10
)

{'ids': [['9183416',
   '9489328',
   '9283931',
   '9467715',
   '9220149',
   '9351005',
   '9225700',
   '9388561',
   '9507765',
   '9396470']],
 'distances': [[4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035,
   4.5503716468811035]],
 'metadatas': [[{'Title': 'santa evita s01 e04 una esposa perfecta 2022'},
   {'Title': 'a gun for jennifer 1997'},
   {'Title': 'blue bloods s13 e03 ghosted 2022'},
   {'Title': 'kita kita 2017'},
   {'Title': 'the pink cloud 2021'},
   {'Title': 'coopers treasure s02 e03 wreck in the shallows 2018'},
   {'Title': 'sudden death 1995'},
   {'Title': 'restaurant to another world s02 e06 rice burgerpizza 2021'},
   {'Title': 'call me fitz s03 e09 teetotal recall 2012'},
   {'Title': 'bad education s04 e02 whodunnit '}]],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None}

In [18]:
query='the last nosferatu 2023'
result=collection.query(
    query_texts = query,
    include=["metadatas", 'distances'],
    n_results=10
)

ids = result['ids'][0]
distances = result['distances'][0]
metadatas = result['metadatas'][0]
zipped_data = zip(ids, distances, metadatas)
sorted_data = sorted(zipped_data, key=lambda x: x[1], reverse=True)
for _, distance, metadata in sorted_data:
    subtitle_name = metadata['Title']
    print(f"Title: {subtitle_name.upper()}")


Title: SANTA EVITA S01 E04 UNA ESPOSA PERFECTA 2022
Title: A GUN FOR JENNIFER 1997
Title: BLUE BLOODS S13 E03 GHOSTED 2022
Title: KITA KITA 2017
Title: THE PINK CLOUD 2021
Title: COOPERS TREASURE S02 E03 WRECK IN THE SHALLOWS 2018
Title: SUDDEN DEATH 1995
Title: RESTAURANT TO ANOTHER WORLD S02 E06 RICE BURGERPIZZA 2021
Title: CALL ME FITZ S03 E09 TEETOTAL RECALL 2012
Title: BAD EDUCATION S04 E02 WHODUNNIT 
